In [21]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] =  userdata.get('OPENAI_API_KEY')

In [22]:
!pip install -q langchain-openai langchain-community langchain-core requests duckduckgo-search

In [23]:
!pip install -U ddgs

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests

In [25]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke('top news in India today')

In [26]:
results

"4 days ago · Get the latest updates and breaking news from India , including politics, elections, government, business, technology, and Bollywood. Check today ' s top 10 news stories, headline news , breaking news , latest news , politics news, sports news, entertainment news and business news on Times of India Read full articles, watch videos, browse thousands of titles and more on the ' India ' topic with Google News. News Headlines Today : Hindustan Times offers the most reliable news in politics, sports, entertainment and business, featuring National and International news Breaking News on Hindustan Times Stay Up-to-date with Top news in India , current headlines, photos & videos online, live coverage on entertainment, business, politics, sports, technology and more at indianexpress.com. How good is India vs India? Overall, India enjoys a favourable head-to-head record against its cross-border rival in the shortest format of the game, having won 11 out of 15 encounters, losing thr

In [27]:
@tool
def get_weather_data(city: str) -> str:
  '''
  This function fecthes the current weather data for a given city
  '''
  url = f'http://api.weatherstack.com/current?access_key=5796f58a8b32915e808b22be509220ab&query={city}'
  response = requests.get(url)

  return response.json()

In [28]:
llm = ChatOpenAI()

In [29]:
llm.invoke('hi')

AIMessage(content='Hello, how can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CKooJ6vtlOAJRC5j9KqadVoMIANxo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--d67c1bb2-a49f-43aa-8365-4c2241f185f4-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [30]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [31]:
# Step 2: Pull the Reacg prompt from the LangChain Hub
prompt = hub.pull("hwchase17/react") # pulls the standard ReAct (Reasoning + Action) agent prompt

In [32]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [33]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)

In [34]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of Karnataka, then find it's current weather condition"})
print(response)



> Entering new AgentExecutor chain...
First, I need to find the capital of Karnataka using a search query. Once I have that information, I can use the get_weather_data function to find the current weather condition of that city.
Action: duckduckgo_search
Action Input: "Capital of Karnataka"Following India's independence in 1947, Bengaluru became the capital of Mysore State, and remained the capital when the state was enlarged and unified in 1956 and subsequently … 4 days ago · The capital is Bengaluru (Bangalore), near the southeastern border. Before the independence of India in 1947, Mysore was a prosperous and progressive but landlocked … Bengaluru, also known as Bangalore, is the capital of Karnataka. It is located in the southeastern part of Karnataka state, on the Deccan Plateau. Bengaluru is famous as a major IT and … May 22, 2024 · Bangalore, officially known as Bengaluru, is a bustling city in the southern part of India and the capital of Karnataka. Bangalore, officially know

In [35]:
response['output']

'The capital of Karnataka is Bangalore, and its current weather condition is partly cloudy with a temperature of 22°C.'